In [ ]:
import numpy as np
from PIL import Image
import os
import requests
from io import BytesIO

import tensorflow as tf
slim = tf.contrib.slim

In [ ]:
IMAGE_SIZE = 299

TENSORFLOW_MODELS_ROOT = '.'
INCEPTION_CHECKPOINT_DIR = '.'
CHECKPOINT_FILE = os.path.join(
    INCEPTION_CHECKPOINT_DIR, 'inception_resnet_v2_2016_08_30.ckpt')

MALAMUTE_PATH = './images/malamute.png'
HUSKY_PATH = './images/Siberian-husky.jpg'

MALAMUTE_IMAGE = Image.open(MALAMUTE_PATH)
HUSKY_IMAGE = Image.open(HUSKY_PATH)

In [ ]:
def convertToJpeg(im):
    with BytesIO() as f:
        im.save(f, format='JPEG')
        return f.getvalue()
    

def create_readable_names_for_imagenet_labels():
    """Create a dict mapping label id to human readable string.
    Returns:
      labels_to_names: dictionary where keys are integers from to 1000
      and values are human-readable names.
    We retrieve a synset file, which contains a list of valid synset labels used
    by ILSVRC competition. There is one synset one per line, eg.
          #   n01440764
          #   n01443537
    We also retrieve a synset_to_human_file, which contains a mapping from synsets
    to human-readable names for every synset in Imagenet. These are stored in a
    tsv format, as follows:
          #   n02119247    black fox
          #   n02119359    silver fox
    We assign each synset (in alphabetical order) an integer, starting from 1
    (since 0 is reserved for the background class).
    """

    # pylint: disable=g-line-too-long
    base_url = 'https://raw.githubusercontent.com/tensorflow/models/master/research/slim/datasets'
    synset_url = '{}/imagenet_lsvrc_2015_synsets.txt'.format(base_url)
    synset_to_human_url = '{}/imagenet_metadata.txt'.format(base_url)

    synset_list = [s.strip() for s in requests.get(synset_url).content.decode().split('\n') if len(s.strip())]
    num_synsets_in_ilsvrc = len(synset_list)
    print(synset_list[-1])
    assert num_synsets_in_ilsvrc == 1000

    synset_to_human_list = requests.get(synset_to_human_url).content.decode().split('\n')[:-1]
    num_synsets_in_all_imagenet = len(synset_to_human_list)
    assert num_synsets_in_all_imagenet == 21842

    synset_to_human = {}
    for s in synset_to_human_list:
        parts = s.strip().split('\t')
        assert len(parts) == 2
        synset = parts[0]
        human = parts[1]
        synset_to_human[synset] = human

    label_index = 1
    labels_to_names = {0: 'background'}
    for synset in synset_list:
        name = synset_to_human[synset]
        labels_to_names[label_index] = name
        label_index += 1

    return labels_to_names

labels_mapping = create_readable_names_for_imagenet_labels()

In [ ]:
from scipy.misc import imresize
from skimage.io import imread, imsave, imshow

%matplotlib inline  
import matplotlib.pyplot as plt

# Increase plot sizes
plt.rcParams['figure.figsize'] = (10, 6)

In [ ]:
import sys
sys.path.append(os.path.join(
    TENSORFLOW_MODELS_ROOT, 'models/research/slim/nets'))
from inception_resnet_v2 import *
import numpy as np

In [ ]:
input_tensor = tf.placeholder(tf.float32, shape=(None,IMAGE_SIZE,IMAGE_SIZE,3), name='input_image')
# NOTE: NO RESCALING!!!

print(input_tensor)
#Load the model
arg_scope = inception_resnet_v2_arg_scope()
with slim.arg_scope(arg_scope):
    logits, end_points = inception_resnet_v2(input_tensor, is_training=False, reuse=None)

sess = tf.Session()
saver = tf.train.Saver()
saver.restore(sess, CHECKPOINT_FILE)

print('====== ENDPOINTS ======')
print(end_points.keys())

In [ ]:
from scipy.ndimage.filters import gaussian_filter

def logit_with_value(tensor, coordinates, value):
    z = tf.zeros_like(tensor[0])
    indices = [coordinates]
    values = [value]
    shape = tf.shape(tensor, out_type=tf.int64)
    delta = tf.SparseTensor(indices, values, shape)
    return z + tf.sparse_tensor_to_dense(delta)


def _prediction_info(predict_values, target_class_index):
    predicted_label = np.argmax(predict_values)
    class_str = "{} ({} prob: {:6f})".format(
        predicted_label, labels_mapping[predicted_label], predict_values[0][predicted_label])
    target_str = "For target class {} : {:6f}".format(
        target_class_index, predict_values[0][target_class_index])
    return "{}. {}".format(class_str, target_str)

def plot_example(class_index, iterations=50, decay = 0.0, norm_pct = 0.0, contrib_pct = 0.0, abs_perc = 0.0,
                 gausian_kernel_size=0.5, gausian_every=4, learning_rate = 2.0):
    im = np.random.normal(size=(IMAGE_SIZE, IMAGE_SIZE, 3), loc=0, scale=1)
    im = np.clip(im, -1, 1).astype(np.float32)
    im = im.reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 3)

    grad_values = logit_with_value(logits, [0, class_index], 1.0)
    grad = tf.gradients(logits, input_tensor, grad_ys=grad_values)

    res = input_tensor + tf.multiply(grad, tf.constant(learning_rate))
    
    modified_input = im
    for i in range(iterations):
        predict_values, modified_image, gradients = sess.run(
            [end_points['Predictions'], res, grad], feed_dict={input_tensor: modified_input})

        if i % 100 == 0:
            print("{}: {}".format(i, _prediction_info(predict_values, class_index)))
        
        regularized = modified_image[0]
        if i % 100 == 99:
            plt.figure()
            plt.suptitle('For {}'.format(i))
            plt.imshow(((modified_input[0] + 1.0) * 0.5 * 255.0).astype(np.uint8))
        if i + 1 != iterations:
            # decay
            regularized = regularized * (1.0 - decay)
            
            # regularize by abs of contribution
            activations = np.abs(regularized* gradients[0]).sum(axis=3) # sum over color channels
            activations = activations[0]
            cutoff = np.percentile(activations, q=contrib_pct)
            temp = activations < cutoff
            selected = np.zeros((1, IMAGE_SIZE, IMAGE_SIZE, 3), dtype=np.bool)
            selected[0, :, :, 0] = temp
            selected[0, :, :, 1] = temp
            selected[0, :, :, 2] = temp
            regularized[selected] = 0
            
            # regularize by norm
            activations = np.linalg.norm(regularized, axis=3) # norm over color channels
            activations = activations[0]
            cutoff = np.percentile(activations, q=norm_pct)
            temp = activations < cutoff
            selected = np.zeros((1, IMAGE_SIZE, IMAGE_SIZE, 3), dtype=np.bool)
            selected[0, :, :, 0] = temp
            selected[0, :, :, 1] = temp
            selected[0, :, :, 2] = temp
            regularized[selected] = 0    

            # regularize by abs
            activations = np.abs(regularized)
            cutoff = np.percentile(activations, q=abs_perc)
            regularized[np.abs(regularized) <= cutoff] = 0
            
            # gausian blur
            if gausian_every != 0 and i % gausian_every == gausian_every - 1:
                for channel in range(3):
                    regularized[:, :, :, channel] = gaussian_filter(regularized[:, :, :, channel], sigma=0.5)
        
        regularized = np.clip(regularized, -1, 1)
                
        modified_input = regularized.astype(np.float32)

    modified_input = modified_input.astype(np.uint8)
    predict_values = sess.run(end_points['Predictions'], feed_dict={input_tensor: modified_input})

    print(_prediction_info(predict_values, class_index))

    return modified_input[0]

In [ ]:
# TARGET_CLASS = 75 # garden spider
TARGET_CLASS = 131 # falmingo
ITERATIONS = 2000
ATTEMPTS = 2

In [ ]:
for _ in range(ATTEMPTS):
    res1 = plot_example(class_index=TARGET_CLASS, iterations=ITERATIONS, decay=0, gausian_kernel_size=0.5, 
                        gausian_every=4, norm_pct=50.0, contrib_pct=0.0, abs_perc=0.0, learning_rate = 1.0)

In [ ]:
for _ in range(ATTEMPTS):
    res2 = plot_example(class_index=TARGET_CLASS, iterations=ITERATIONS, decay=0.03, gausian_kernel_size=1.0, 
                        gausian_every=0, norm_pct=15.0, contrib_pct=0.0, abs_perc=0.0, learning_rate = 1.0)

In [ ]:
for _ in range(ATTEMPTS):
    res3 = plot_example(class_index=TARGET_CLASS, iterations=ITERATIONS, decay=0.0001, gausian_kernel_size=0.7, 
                        gausian_every=4, norm_pct=25.0, contrib_pct=25.0, abs_perc=0.0, learning_rate = 2.0)

In [ ]:
for _ in range(ATTEMPTS):
    res4 = plot_example(class_index=TARGET_CLASS, iterations=ITERATIONS, decay=0, gausian_kernel_size=0.5, 
                        gausian_every=4, norm_pct=0.0, contrib_pct=50, abs_perc=0.0, learning_rate = 1.0)